<a href="https://colab.research.google.com/github/DATASCIENCE-PROJECT-GROUP/dspl/blob/main/DSPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRAIN.CSV - PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from scipy import sparse

In [2]:
# Define chunk size to reduce memory usage
chunk_size = 10000  # Adjust based on available memory
use_cols = ["Customer_ID", "outlet_city", "luxury_sales", "fresh_sales", "dry_sales", "cluster_catgeory"]

In [3]:
def process_chunk(chunk):
    # Select necessary columns
    chunk = chunk[use_cols]

In [5]:
def process_chunk(chunk):
    # Select necessary columns
    chunk = chunk[use_cols]
    # Reduce memory usage by downcasting numerical columns
    for col in chunk.select_dtypes(include=['int64']).columns:
        chunk[col] = pd.to_numeric(chunk[col], downcast='integer')
    for col in chunk.select_dtypes(include=['float64']).columns:
        chunk[col] = pd.to_numeric(chunk[col], downcast='float')

    return chunk

In [6]:
# Read dataset in chunks and process each chunk
chunks = []
for chunk in pd.read_csv("/content/traincsv.csv", chunksize=chunk_size, usecols=use_cols):
    chunks.append(process_chunk(chunk))

In [7]:
df = pd.concat(chunks, ignore_index=True)

In [8]:
# Handling missing values
imputer = SimpleImputer(strategy="median")
numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

In [9]:
# Encoding categorical variables with sparse matrices
categorical_features = df.select_dtypes(include=["object"]).columns

In [10]:
# Ensure categorical values are strings
df[categorical_features] = df[categorical_features].astype(str)

In [11]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True, dtype=np.float32)
categorical_transformed = encoder.fit_transform(df[categorical_features])

In [12]:
df = df.drop(columns=categorical_features)

In [13]:
# Convert sparse matrix to a sparse DataFrame
df_encoded = pd.DataFrame.sparse.from_spmatrix(categorical_transformed, columns=encoder.get_feature_names_out())
df = pd.concat([df, df_encoded], axis=1)

In [ ]:
# Splitting data before scaling to save memory
X = df.drop(columns=df.columns[df.columns.str.startswith('x0_')])  # Dropping the encoded categorical columns
y = df["Customer_ID"] # Using Customer_ID as target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)